In [1]:
from dotenv import load_dotenv
import os
import requests
import plaid
import datetime
import json

In [2]:
load_dotenv()

True

In [3]:
api_key0 = os.getenv("QUANDL_API_KEY")
api_key1 = os.getenv("ALPACA_API_KEY")
api_key2 = os.getenv("ALPACA_SECRET_KEY")

plaid_client_id = os.getenv("PLAID_CLIENT_ID")
plaid_public_key = os.getenv("PLAID_PUBLIC_KEY")
plaid_sbx_secret_key = os.getenv("PLAID_SBX_SECRET_KEY")

type(plaid_client_id)
type(plaid_public_key)
type(plaid_sbx_secret_key)

str

In [18]:
# concatenate request_url with api_key variable
request_url = "https://www.quandl.com/api/v3/datasets/WIKI/MSFT.json?api_key="
request_url = request_url + api_key0
request_url

'https://www.quandl.com/api/v3/datasets/WIKI/MSFT.json?api_key=VJHGDysWB2K13tDXac-V'

In [19]:
request_data = requests.get(request_url)
# request_data.content

In [33]:
# PLAID PRACTICE

# Extract list of institutions from Plaid
# Create client object, in order to send requests to plaid, it needs a client object
client = plaid.Client(client_id = plaid_client_id, secret = plaid_sbx_secret_key, 
                     public_key = plaid_public_key, environment = 'sandbox')

client.Institutions.get(2)

{'institutions': [{'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112060',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Bank (Broadus, MT) - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',
    'balance',
    'transactions',
    'income',
    'identity'],
   'routing_numbers': []},
  {'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112062',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Constitution Bank - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',


In [36]:
# Create public token and exchange for access token
# Select an institution for processing, can retrieve the name of the institution
INSTITUTION_ID = "ins_109512"

# Create public token to be exchanged for institution access token
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions', 'income', 'assets'])

# Exchange public token for access token
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Store access token as variable
access_token = exchange_response['access_token']


In [46]:
# Get list of Accounts
client.Accounts.get(access_token)

# Parse the JSON output, and store the access token as a Python variable.
# Extract Transactions with date range
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-365))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now()) 

# Execute get request for transactions
transaction_response = client.Transactions.get(access_token, start_date, end_date)

# print json output

# print(json.dumps(transaction_response['transactions'][:2], indent=4, sort_keys=True))

for transaction in transaction_response['transactions']: 
    if transaction['name'] == 'INTRST PYMNT': 
        print(json.dumps(transaction['amount'], indent=4, sort_keys=True))

-4.22
-4.22
-4.22
-4.22
-4.22
-4.22
